<a href="https://colab.research.google.com/github/bdi2357/CoverLetter/blob/main/CoverLetrrer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [3]:
!git clone https://github.com/bdi2357/CoverLetter

Cloning into 'CoverLetter'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (21/21), 81.22 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
!pip install -q PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [5]:
!pip install docx
!pip install fpdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53894 sha256=b161102b556b4b2ed66d729eaad968a253aa1101b16664114086b0cdf3c3bc35
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=d95cf470e7a6877833f929167b2f340968a6f0130403df7744270693223817ae
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [7]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


In [8]:
#from genai import GenerativeModel
from docx import Document
from docx.shared import Pt
import re
import time

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [1]:
def generate_cover_letter(model,cv_text, job_description_text):

    prompt = f"""Based on the provided job description and CV, write a compelling and personalized cover letter that highlights relevant skills and experiences.

    Job Description:\n{job_description_text}\n\nCV:\n{cv_text}\n\n

    Tailor the letter to the specific job requirements and showcase the candidate's enthusiasm for the position. Be sure to mention specific accomplishments and quantify results whenever possible. Keep the tone professional and engaging.
    As concise as possible no more than 4 sentences. Avoid using adjectives unless neccessary , as less as possible , do not use emotive adjectives in any case, use only factual information based on the cv_text"""
    print("HERE")
    response = model.generate_content(prompt)  # Remove the 'context' argument
    return response